# MVSep MDX23 Colab
Colab version of MDX23 algorithm from [MVSEP.COM](https://www.mvsep.com) with some tweaks:
* updated with new voc_ft & UVR-MDX-Instr-HQ3 models
* folder batch processing (all audio files inside provided input folder will be separated; don't paste filenames in this Colab - only folder paths allowed)
* fixed high frequency bleed in vocals
* fixed volume compensation for MDX models
* it receives input files from temp Colab disk by default here, not GDrive (open file manager in the left to upload a file), but you can change it to GDrive by setting /content/drive/MyDrive/input 
* For a bit better quality, chunks_size can be set to 500K for ~2:41-3:46(?) tracks - 500K fails with 5 minute tracks
* Both overlaps set to 0.8 is (still) a good balance between duration and quality, but slower
* If you use 48kHz file instead of 44kHz, it will show a lot of warinings (e.g. PySoundFile failed), but separation will be successful

Credits:
* [https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Models by [Demucs](https://github.com/facebookresearch/demucs), [UVR GUI Team](https://github.com/Anjok07/ultimatevocalremovergui) - Anjok / Aufr33 & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation & tweaks by [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)

In [ ]:
#@markdown #Installation
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take 1 minute...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/deton24/MVSEP-MDX23-Colab_v2.1 &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2.1
!pip install -r requirements.txt &> /dev/null
print('Installation done !')

In [ ]:
#@markdown #Separation
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2.1
def console(t):
    get_ipython().system(t)

#file_path = '/content/drive/MyDrive/mvsep_dataset/' #@param {type:"string"}
folder_path = '/content/MVSEP-MDX23-Colab_v2.1/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
overlap_large = 0.6 #@param {type:"slider", min:0.1, max:1, step:0.05}
overlap_small = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown *Use lower chunk_size if you have memory errors*
chunk_size = 300000 #@param {type:"slider", min:100000, max:1000000, step:100000}

#console(f'python inference.py --large_gpu --chunk_size {chunk_size} --input_audio "{file_path}" --output_folder "./results/{filename}"')
for file_path in glob.glob(folder_path+"/*"):
  filename =  Path(file_path).stem
  Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
  console(f'python inference.py --large_gpu --chunk_size {chunk_size} --input_audio "{file_path}" --overlap_large {overlap_large} --overlap_small {overlap_small} --output_folder "{output_folder}"/"{filename}"')


In [ ]:
#@title (optional) Mixdown to 64 bit (might be slightly better instrum2 equivalent (32-bit float) - can be placebo in most cases; change it to pcm_s16le for 16 bit). Write file name without input file extension in paths below
!ffmpeg -i "/content/drive/MyDrive/output/your track/your track_bass.wav" -i "/content/drive/MyDrive/output/your track/your track_other.wav" -i "/content/drive/MyDrive/output/your track/your track_drums.wav" -filter_complex "[0]volume=3[a];[1]volume=3[b];[2]volume=3[c];[a][b][c]amix=inputs=3:duration=longest" -c:a pcm_f64le '/content/drive/MyDrive/output/output.wav'